# 알약 판별 EDA

In [2]:
# 노트북 상위 폴더 경로 추가
import sys
import os
sys.path.append(os.path.abspath(".."))

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import glob
from collections import Counter
import pandas as pd


# 유틸리티 함수 임포트
from src.utils import get_device, init_logger

In [4]:
logger = init_logger(name="healtheat_vision")  # 여기서만 프로젝트 이름을 지정
device = get_device()

logger.info(f"Using device: {device}")

2025-12-12 16:00:49 [INFO] (600008572.py:4) - Using device: mps


Apple MPS GPU detected.


In [ ]:
from pathlib import Path
import json
import pprint

root = Path("../data")
img_dir = root / "train_images"
ann_dir = root / "train_annotations"
ann_aihub_dir = root / "aihub_downloads"

if not img_dir.exists() or not ann_dir.exists() or not ann_aihub_dir.exists():
    logger.error("경로가 존재하지 않습니다. 데이터 폴더를 확인하세요.")
else:
    logger.info("데이터 경로 확인 완료.")

# 2) 루트 경로의 폴더/파일 목록 로깅
folders = [p.name for p in root.iterdir() if p.is_dir()]
files   = [p.name for p in root.iterdir() if p.is_file()]

logger.info(
    f"Paths OK\n"
    f"  - img_dir : {img_dir.resolve()}\n"
    f"  - ann_dir : {ann_dir.resolve()}\n"
    f"  - ann_aihub_dir : {ann_aihub_dir.resolve()}"
)

logger.info(
    "Root structure:\n"
    f"  - Folders: {folders}\n"
    f"  - Files  : {files}"
)

2025-12-12 16:00:49 [INFO] (394718001.py:13) - 데이터 경로 확인 완료.
2025-12-12 16:00:49 [INFO] (394718001.py:19) - Paths OK
  - img_dir : /Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/data/train_images
  - ann_dir : /Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/data/train_annotations
  - ann_aihub_dir : /Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/data/aihub_downloads
2025-12-12 16:00:49 [INFO] (394718001.py:26) - Root structure:
  - Folders: ['test_images', 'train_annotations', 'aihub_downloads', 'eda_files', 'train_images', 'train_labels']
  - Files  : ['.DS_Store', 'train.txt', '.gitkeep', 'missing_annotations.txt', 'val.txt']


In [6]:
# 1) 이미지 경로 수집
img_paths = list(img_dir.glob("*.png"))
print(f"총 이미지 수(실제 PNG): {len(img_paths)}")

# 2) 어노테이션 경로 수집 (재귀)
ann_paths = list(ann_dir.rglob("*.json"))
print(f"총 어노테이션 파일 수(JSON): {len(ann_paths)}")

# 3) ai_hub 어노테이션 경로 수집
ann_aihub_paths = list(ann_aihub_dir.rglob("*.json"))
print(f"총 ai_hub 어노테이션 파일 수(JSON): {len(ann_aihub_paths)}\n")

# 박스 수
total_boxes = 0
for path in ann_paths:
    data = json.loads(path.read_text())
    total_boxes += len(data["annotations"])
print("JSON의 annotations 엔트리 합계", total_boxes)

# JSON 안에서 참조하는 이미지
total_image_entries = 0
json_image_names = set()

for path in ann_paths:
    data = json.loads(path.read_text())
    imgs = data["images"]
    total_image_entries += len(imgs)
    for img_info in imgs:
        json_image_names.add(img_info["file_name"])

print("JSON의 이미지 엔트리 합계:", total_image_entries)
print("JSON에서 참조하는 고유 이미지 수:", len(json_image_names))

총 이미지 수(실제 PNG): 651
총 어노테이션 파일 수(JSON): 1001
총 ai_hub 어노테이션 파일 수(JSON): 40547

JSON의 annotations 엔트리 합계 1001
JSON의 이미지 엔트리 합계: 1001
JSON에서 참조하는 고유 이미지 수: 369


In [7]:
# 3) stem 기반 매핑(dict 생성)
img_dict = {p.stem: p for p in img_paths}
ann_dict = {p.stem: p for p in ann_paths}
ann_aihub_dict = {p.stem: p for p in ann_aihub_paths}

# 4) 공통 key = 정상 매칭된 pair
common_keys = sorted(img_dict.keys() & ann_dict.keys())
print(f"매칭된 이미지-어노테이션 pair 수: {len(common_keys)}")

example_key = common_keys[0]
print("예시 key:", example_key)
print("이미지 경로:", img_dict[example_key])
print("어노테이션 경로:", ann_dict[example_key])


common_keys_aihub = sorted(img_dict.keys() & ann_aihub_dict.keys())
print(f"매칭된 이미지-어노테이션 pair 수: {len(common_keys_aihub)}")

example_key_aihub = common_keys_aihub[0]
print("예시 key:", example_key_aihub)
print("이미지 경로:", img_dict[example_key_aihub])
print("어노테이션 경로:", ann_aihub_dict[example_key_aihub])

매칭된 이미지-어노테이션 pair 수: 232
예시 key: K-001900-016548-019607-029451_0_2_0_2_70_000_200
이미지 경로: ../data/train_images/K-001900-016548-019607-029451_0_2_0_2_70_000_200.png
어노테이션 경로: ../data/train_annotations/K-001900-016548-019607-029451_json/K-019607/K-001900-016548-019607-029451_0_2_0_2_70_000_200.json
매칭된 이미지-어노테이션 pair 수: 651
예시 key: K-001900-010224-016551-031705_0_2_0_2_70_000_200
이미지 경로: ../data/train_images/K-001900-010224-016551-031705_0_2_0_2_70_000_200.png
어노테이션 경로: ../data/aihub_downloads/raw_extracted/K-001900-010224-016551-031705_json/K-016551/K-001900-010224-016551-031705_0_2_0_2_70_000_200.json


In [8]:
# JSON 구조 확인 (1개만)
sample_json = json.loads(ann_dict[example_key].read_text())
print(f"Annotation File: {ann_dict[example_key].name}")
print("\nJSON Key 구조:", sample_json.keys())
pprint.pp(sample_json)

Annotation File: K-001900-016548-019607-029451_0_2_0_2_70_000_200.json

JSON Key 구조: dict_keys(['images', 'type', 'annotations', 'categories'])
{'images': [{'file_name': 'K-001900-016548-019607-029451_0_2_0_2_70_000_200.png',
             'width': 976,
             'height': 1280,
             'imgfile': 'K-001900-016548-019607-029451_0_2_0_2_70_000_200.png',
             'drug_N': 'K-019607',
             'drug_S': '정상알약',
             'back_color': '연회색 배경',
             'drug_dir': '앞면',
             'light_color': '주백색',
             'camera_la': 70,
             'camera_lo': 0,
             'size': 200,
             'dl_idx': '19606',
             'dl_mapping_code': 'K-019607',
             'dl_name': '스토가정 10mg',
             'dl_name_en': 'Stogar Tab. 10mg',
             'img_key': 'http://connectdi.com/design/img/drug/1MrFeLzYHjR.jpg',
             'dl_material': '라푸티딘',
             'dl_material_en': 'Lafutidine',
             'dl_custom_shape': '정제, 저작정',
             'dl_com

In [9]:
# aihub_JSON 구조 확인 (1개만)
sample_json = json.loads(ann_aihub_dict[example_key].read_text())
print(f"Annotation File: {ann_aihub_dict[example_key].name}")
print("\nJSON Key 구조:", sample_json.keys())
pprint.pp(sample_json)

Annotation File: K-001900-016548-019607-029451_0_2_0_2_70_000_200.json

JSON Key 구조: dict_keys(['images', 'type', 'annotations', 'categories'])
{'images': [{'file_name': 'K-001900-016548-019607-029451_0_2_0_2_70_000_200.png',
             'width': 976,
             'height': 1280,
             'imgfile': 'K-001900-016548-019607-029451_0_2_0_2_70_000_200.png',
             'drug_N': 'K-019607',
             'drug_S': '정상알약',
             'back_color': '연회색 배경',
             'drug_dir': '앞면',
             'light_color': '주백색',
             'camera_la': 70,
             'camera_lo': 0,
             'size': 200,
             'dl_idx': '19606',
             'dl_mapping_code': 'K-019607',
             'dl_name': '스토가정 10mg',
             'dl_name_en': 'Stogar Tab. 10mg',
             'img_key': 'http://connectdi.com/design/img/drug/1MrFeLzYHjR.jpg',
             'dl_material': '라푸티딘',
             'dl_material_en': 'Lafutidine',
             'dl_custom_shape': '정제, 저작정',
             'dl_com

In [22]:
# img ↔ aihub annotation 이 둘 다 존재하는 key
common_keys_aihub = sorted(img_dict.keys() & ann_aihub_dict.keys())
print(f"매칭된 이미지-aihug 어노테이션 pair 수: {len(common_keys_aihub)}")

# 매칭된 aihub annotation dict
matched_ann_aihub_dict = {
    k: ann_aihub_dict[k]
    for k in common_keys_aihub
}

매칭된 이미지-aihug 어노테이션 pair 수: 651


In [23]:
k = common_keys_aihub[0]
print("key:", k)
print("image:", img_dict[k])
print("aihug ann:", matched_ann_aihub_dict[k])

key: K-001900-010224-016551-031705_0_2_0_2_70_000_200
image: ../data/train_images/K-001900-010224-016551-031705_0_2_0_2_70_000_200.png
aihug ann: ../data/aihub_downloads/raw_extracted/K-001900-010224-016551-031705_json/K-016551/K-001900-010224-016551-031705_0_2_0_2_70_000_200.json


In [25]:
from collections import Counter, defaultdict

# 1) aihub / img stem 중복 체크
img_stem_cnt = Counter([p.stem for p in img_paths])
aih_stem_cnt = Counter([p.stem for p in ann_aihub_paths])

dup_img  = [k for k,v in img_stem_cnt.items() if v > 1]
dup_aih  = [k for k,v in aih_stem_cnt.items() if v > 1]

print("중복 stem (images):", len(dup_img))
print("중복 stem (aihub json):", len(dup_aih))

if dup_aih:
    k = dup_aih[0]
    print("\n예시 중복 aihub stem:", k, "개수:", aih_stem_cnt[k])
    for p in [x for x in ann_aihub_paths if x.stem == k][:10]:
        print(" -", p)

중복 stem (images): 0
중복 stem (aihub json): 10506

예시 중복 aihub stem: K-005391-018357-021325_0_2_0_2_90_000_200 개수: 3
 - ../data/aihub_downloads/raw_extracted/K-005391-018357-021325_json/K-018357/K-005391-018357-021325_0_2_0_2_90_000_200.json
 - ../data/aihub_downloads/raw_extracted/K-005391-018357-021325_json/K-005391/K-005391-018357-021325_0_2_0_2_90_000_200.json
 - ../data/aihub_downloads/raw_extracted/K-005391-018357-021325_json/K-021325/K-005391-018357-021325_0_2_0_2_90_000_200.json


In [28]:
from pathlib import Path
import shutil

src_root = Path(ann_aihub_dir)
dst_root = Path(
    "/Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/healtheat_vision/data/aihub_downloads/raw_matched"
)

dst_root.mkdir(parents=True, exist_ok=True)

# 매칭된 json 경로들
src_json_paths = [ann_aihub_dict[k] for k in common_keys_aihub]

# aihub_downloads 기준 '최상위 폴더' set
top_level_dirs = set()

for p in src_json_paths:
    rel = p.relative_to(src_root)
    top_dir = rel.parts[0]   # 👈 핵심
    top_level_dirs.add(top_dir)

print("복사 대상 최상위 폴더 수:", len(top_level_dirs))

copied = 0
skipped = 0

for d in sorted(top_level_dirs):
    src_dir = src_root / d
    dst_dir = dst_root / d

    if dst_dir.exists():
        skipped += 1
        continue

    shutil.copytree(src_dir, dst_dir)
    copied += 1

print("폴더 복사 완료")
print("복사된 폴더:", copied)
print("이미 존재해 스킵:", skipped)



복사 대상 최상위 폴더 수: 1
폴더 복사 완료
복사된 폴더: 0
이미 존재해 스킵: 1


In [21]:
from collections import Counter
import json
import pandas as pd

# aihub 기준으로 "매칭된 pair"만 집계 (원하면 전체도 가능)
keys_to_use = common_keys_aihub   # ✅ 이게 포인트 (aihub 기준)

dl_idx_counter = Counter()
dl_name_counter = Counter()
pair_counter = Counter()          # (dl_idx, dl_name) 쌍
dl_idx_to_name = {}              # dl_idx -> dl_name (대표값)

missing = 0
multi_image_json = 0

for key in keys_to_use:
    p = ann_aihub_dict[key]       # ✅ aihub json 사용
    data = json.loads(p.read_text())

    imgs = data.get("images", [])
    if not imgs:
        missing += 1
        continue

    if len(imgs) > 1:
        multi_image_json += 1

    # 보통 1개지만, 안전하게 images 전체 순회
    for img in imgs:
        dl_idx  = img.get("dl_idx")
        dl_name = img.get("dl_name")

        if not dl_idx or not dl_name:
            missing += 1
            continue

        dl_idx_counter[dl_idx] += 1
        dl_name_counter[dl_name] += 1
        pair_counter[(dl_idx, dl_name)] += 1

        # 같은 dl_idx가 다른 name을 갖는 케이스 체크용으로도 쓸 수 있음
        dl_idx_to_name.setdefault(dl_idx, dl_name)

print("=== 요약(aihug json + 매칭 pair 기준) ===")
print("고유 dl_idx 수:", len(dl_idx_counter))
print("고유 dl_name 수:", len(dl_name_counter))
print("고유 (dl_idx, dl_name) 쌍 수:", len(pair_counter))
print("images 누락/필드 누락 건수(대략):", missing)
print("images가 2개 이상인 json 수:", multi_image_json)

=== 요약(aihug json + 매칭 pair 기준) ===
고유 dl_idx 수: 32
고유 dl_name 수: 32
고유 (dl_idx, dl_name) 쌍 수: 32
images 누락/필드 누락 건수(대략): 0
images가 2개 이상인 json 수: 0


In [ ]:
import json
import os
import glob
from collections import Counter
import pandas as pd

# dl_idx / dl_name 카운트
dl_idx_counter = Counter()
dl_idx_to_name = {}

for p in aihub_jsons:
    with open(p, "r") as f:
        data = json.load(f)

    # images 메타데이터 기준
    for img in data.get("images", []):
        dl_idx  = img.get("dl_idx")
        dl_name = img.get("dl_name")

        if dl_idx is None or dl_name is None:
            continue

        dl_idx_counter[dl_idx] += 1
        dl_idx_to_name[dl_idx] = dl_name

In [20]:
print(f"고유 dl_idx 개수: {len(dl_idx_counter)}")
print("고유 dl_idx 일부:", list(dl_idx_counter.keys())[:20])

고유 dl_idx 개수: 56
고유 dl_idx 일부: ['3350', '16231', '33879', '41767', '35205', '29666', '13899', '20013', '16687', '16547', '1899', '29450', '19606', '22073', '3831', '27925', '21770', '36636', '18146', '16261']


In [ ]:
from collections import Counter

# ======================
# dl_idx / dl_name 집계
# ======================

dl_idx_counter  = Counter()
dl_name_counter = Counter()
pair_counter    = Counter()  # (id, name) 쌍 기준

for key in common_keys:
    ann_path = ann_dict[key]
    data = json.loads(ann_path.read_text())

    # 구조상 images는 리스트지만 보통 1개
    img_info = data["images"][0]

    dl_idx  = img_info.get("dl_idx")
    dl_name = img_info.get("dl_name")

    if dl_idx is None or dl_name is None:
        continue

    dl_idx_counter[dl_idx] += 1
    dl_name_counter[dl_name] += 1
    pair_counter[(dl_idx, dl_name)] += 1

In [18]:
print("=== 요약 ===")
print("고유 dl_idx 수 :", len(dl_idx_counter))
print("고유 dl_name 수:", len(dl_name_counter))
print("고유 (dl_idx, dl_name) 쌍 수:", len(pair_counter))

=== 요약 ===
고유 dl_idx 수 : 25
고유 dl_name 수: 25
고유 (dl_idx, dl_name) 쌍 수: 25


In [ ]:
# ============================================
# [A] 어노테이션 전체에서 등장하는 고유 클래스 (ID + 이름) 리스트 + 카운트
# ============================================

import json
import os
import glob
from collections import Counter
import pandas as pd

root    = "../data"
ann_dir = os.path.join(root, "train_annotations")

# 모든 json 파일 불러오기
ann_paths = glob.glob(os.path.join(ann_dir, "**", "*.json"), recursive=True)

# category_id -> name 매핑
category_id_to_name = {}
# 클래스 카운트는 "ID" 기준으로 쌓는다
class_counter = Counter()

for p in ann_paths:
    with open(p, "r") as f:
        data = json.load(f)

    # categories 에서 id -> name 매핑 갱신
    for cat in data["categories"]:
        category_id_to_name[cat["id"]] = cat["name"]

    # annotations 기준으로 category_id 카운트
    for ann in data["annotations"]:
        cid = ann["category_id"]
        class_counter[cid] += 1


# ============================================
# [B] 고유 클래스 리스트 (ID + 이름) 추출
# ============================================

unique_category_ids = sorted(class_counter.keys())
num_unique_classes  = len(unique_category_ids)

print(f"고유 클래스 개수: {num_unique_classes}")
print("고유 category_id 일부:", unique_category_ids[:20])

# id + name 매핑 리스트
unique_classes = [
    {
        "category_id": cid,
        "class_name": category_id_to_name.get(cid, f"id_{cid}")
    }
    for cid in unique_category_ids
]

df_unique_classes = pd.DataFrame(unique_classes)
print(df_unique_classes.head())


# ============================================
# [C] 클래스별 카운트 DataFrame (ID + 이름 + count)
# ============================================

rows = []
for cid, cnt in class_counter.items():
    rows.append({
        "category_id": cid,
        "class_name": category_id_to_name.get(cid, f"id_{cid}"),
        "count": cnt,
    })

df_class_counts = (
    pd.DataFrame(rows)
    .sort_values("count", ascending=False)
    .reset_index(drop=True)
)

print(df_class_counts.head())


# ============================================
# [D] CSV로 저장
# ============================================

# csv_unique_path = "annotation_unique_classes_with_id.csv"
# csv_count_path  = "annotation_class_counts_with_id.csv"

# # (1) 고유 클래스 리스트 (ID + 이름)
# df_unique_classes.to_csv(
#     csv_unique_path, index=False, encoding="utf-8-sig"
# )

# # (2) 클래스별 카운트 (ID + 이름 + count)
# df_class_counts.to_csv(
#     csv_count_path, index=False, encoding="utf-8-sig"
# )

# print(f"[완료] 고유 클래스(ID+이름) 리스트 CSV → {csv_unique_path}")
# print(f"[완료] 클래스별 카운트(ID+이름+count) CSV → {csv_count_path}")

고유 클래스 개수: 56
고유 category_id 일부: [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4542, 12080, 12246, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18146, 18356]
   category_id          class_name
0         1899          보령부스파정 5mg
1         2482         뮤테란캡슐 100mg
2         3350         일양하이트린정 2mg
3         3482  기넥신에프정(은행엽엑스)(수출용)
4         3543   무코스타정(레바미피드)(비매품)
   category_id          class_name  count
0         3350         일양하이트린정 2mg    240
1         3482  기넥신에프정(은행엽엑스)(수출용)     45
2        35205        아토젯정 10/40mg     40
3        16261          크레스토정 20mg     31
4        20237          플라빅스정 75mg     29
[완료] 고유 클래스(ID+이름) 리스트 CSV → annotation_unique_classes_with_id.csv
[완료] 클래스별 카운트(ID+이름+count) CSV → annotation_class_counts_with_id.csv


In [15]:
class_counts = Counter()

for key in common_keys:
    for ann in image_to_anns[key]:
        cid = ann["category_id"]
        cls_name = category_id_to_name.get(cid, f"id_{cid}")
        class_counts[cls_name] += 1

print("클래스 개수:", len(class_counts))
for cls, cnt in class_counts.most_common(30):
    print(f"{cls:30s} : {cnt}")

NameError: name 'image_to_anns' is not defined

## 어노테이션 기준 데이터 프레임